# 🤖 ContractCopilot: Multi-Agent AI Contract Review System

This notebook demonstrates how to use **CrewAI** to build a smart, multi-agent system for **automated contract review**.

We simulate a real-world scenario where:
1. A contract document is processed.
2. Multiple specialized agents extract, assess, and summarize legal insights.
3. A final email report is prepared for the client.

🚀 Technologies Used:
- Python
- [CrewAI](https://docs.crewai.com/) for multi-agent orchestration
- PyPDF2 (if using PDFs)
- Local/hosted LLMs (Ollama, OpenAI, or similar)

📌 Ideal For:
- Legal teams
- Freelancers and HR departments
- AI Automation Portfolios (for LinkedIn/GitHub)

In [1]:
# Install CrewAI and PyPDF2 (if not already installed)
!pip install crewai PyPDF2 --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.3/265.3 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 62.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.3/135.3 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 62.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 57.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 5.2 MB/s eta 0:

In [3]:
from crewai import Agent, Task, Crew, LLM
from PyPDF2 import PdfReader  # Optional if using PDF

In [4]:
import os
from google.colab import userdata
os.environ["GEMINI_API_KEY"] = userdata.get('GEMINI_API_KEY')

my_llm = LLM(
              model='gemini/gemini-1.5-flash',
              api_key=os.environ["GEMINI_API_KEY"]
            )

## 🧠 What is CrewAI?

CrewAI lets you build workflows with multiple AI agents that collaborate on complex tasks.

Each agent:
- Has a defined **role** and **goal**
- Handles a specific **task**
- Communicates with other agents

In this notebook, we'll create 4 agents:
1. **Contract Reader Agent** – Summarizes the contract
2. **Risk Assessor Agent** – Identifies risky/vague clauses
3. **Recommendation Agent** – Suggests improvements using internal policy
4. **Email Agent** – Prepares a final email report for the client

## 📄 Step 4: Load or Simulate a Contract

In a real-world app, you’d extract contract text from a PDF or DOCX file.
For this demo, we simulate a contract below.

In [5]:
# Simulated contract text
contract_text = """
This Service Agreement is made effective as of January 1, 2023, by and between Acme Corp ("Provider") and Beta Ltd ("Client").

1. Scope of Services: Provider agrees to provide digital marketing services.
2. Term: The contract shall begin on the date above and remain in effect until terminated.
3. Payment: Client shall pay $5000 per month, due within 15 days of invoicing.
4. Confidentiality: Both parties agree to confidentiality for 2 years post termination.
5. Termination: Either party may terminate with 30 days' notice, without cause.
"""


## 🛡️ Step 5: Define Internal Policy

We’ll compare contract clauses against your internal rules or compliance policy.


In [6]:
policy_text = """
- Contracts must have a clear termination clause with a minimum 60-day notice.
- Payment terms must be 10 days or less.
- Confidentiality should extend for at least 3 years after termination.
"""


## 🤖 Step 6: Define AI Agents

Each agent has:
- A role (title)
- A goal (their main function)
- A backstory (context/personality for the LLM)

We’ll define four agents below.


In [15]:
reader_agent = Agent(
    role="Contract Reader",
    goal="Extract and summarize the main clauses of the contract",
    backstory="You're a legal expert who reads contracts and explains key parts clearly.",
    llm = my_llm,
    verbose=True
)


In [16]:
assessor_agent = Agent(
    role="Risk Assessor",
    goal="Identify vague or risky contract clauses",
    backstory="You're an expert in legal compliance and risk evaluation.",
    llm = my_llm,
    verbose=True
)


In [17]:
recommender_agent = Agent(
    role="Recommendation Agent",
    goal="Suggest contract edits based on our internal policy",
    backstory="You understand how to improve contracts based on internal legal guidelines.",
    llm = my_llm,
    verbose=True
)


In [18]:
email_agent = Agent(
    role="Email Agent",
    goal="Prepare a professional summary email of the review",
    backstory="You're a polite executive assistant crafting summaries for clients.",
    llm = my_llm,
    verbose=True
)


## 🧩 Step 7: Define Tasks for Agents

Each agent will perform a task in sequence and pass the result forward.
Let’s define these tasks now.


In [19]:
task1 = Task(
    description=f"Summarize the following contract:\n\n{contract_text}",
    expected_output="Bullet-point summary of all clauses.",
    agent=reader_agent
)


In [20]:
task2 = Task(
    description="Using the summary, find any risky, vague, or unclear clauses.",
    context=[task1],
    expected_output="List of risky or vague clauses with explanation.",
    agent=assessor_agent
)


In [21]:
task3 = Task(
    description=f"Compare with this policy:\n\n{policy_text}\n\nSuggest contract improvements.",
    context=[task1, task2],
    expected_output="Recommended edits and reasons.",
    agent=recommender_agent
)


In [22]:
task4 = Task(
    description="Create a client-facing email summarizing the findings.",
    context=[task1, task2, task3],
    expected_output="A professional email with a summary, risks, and suggestions.",
    agent=email_agent
)


## 🧠 Step 8: Create Crew and Run Tasks

Let’s now link all agents and tasks into a workflow using `Crew`.


In [23]:
crew = Crew(
    agents=[reader_agent, assessor_agent, recommender_agent, email_agent],
    tasks=[task1, task2, task3, task4],
    verbose=True
)

result = crew.kickoff()


╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: ffb91fd8-4c1b-460b-859b-4a0da0e5ab78                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: 168277b4-c7df-4e15-81c6-ef9e79e50b4f
       Status: Executing Task...

🚀 Crew: crew
└── 📋 Task: 168277b4-c7df-4e15-81c6-ef9e79e50b4f
       Status: Executing Task...
    └── 🤖 Agent: Contract Reader
            Status: In Progress

# Agent: Contract Reader
## Task: Summarize the following contract:


This Service Agreement is made effective as of January 1, 2023, by and between Acme Corp ("Provider") and Beta Ltd ("Client").

1. Scope of Services: Provider agrees to provide digital marketing services.
2. Term: The contract shall begin on the date above and remain in effect until terminated.
3. Payment: Client shall pay $5000 per month, due within 15 days of invoicing.
4. Confidentiality: Both parties agree to confidentiality for 2 years post termination.
5. Termination: Either party may terminate with 30 days' notice, without cause.



🚀 Crew: crew
└── 📋 Task: 168277b4-c7df-4e15-81c6-ef9e79e50b4f
       Status: Executing Task...
    └── 🤖 Agent: Contract Reader
            Status: In Progress
        └── 🧠 Thinking...

...



# Agent: Email Agent
## Final Answer: 
Subject: Summary of Contract Review and Recommendations - Acme Corp and Beta Ltd.

Dear Beta Ltd.,

This email summarizes our review of the contract between Acme Corp ("Provider") and Beta Ltd ("Client"), effective January 1, 2023, concerning the provision of digital marketing services.  Our review identified several areas requiring immediate attention to mitigate potential risks and ensure compliance with internal policies.

**Summary of Findings:**

The current contract lacks sufficient detail and clarity in several key areas:

* **Vague Service Description:** The absence of specific deliverables and performance metrics for the digital marketing services creates significant uncertainty regarding the scope of work, making it susceptible to scope creep and disputes over payment and performance.  The current $5,000 monthly fee is not tied to quantifiable deliverables.

* **Indefinite Contract Term:** The ongoing nature of the contract without a d

🚀 Crew: crew
├── 📋 Task: 168277b4-c7df-4e15-81c6-ef9e79e50b4f
│      Assigned to: Contract Reader
│      Status: ✅ Completed
│   └── 🤖 Agent: Contract Reader
│           Status: ✅ Completed
├── 📋 Task: 782fab62-6d2a-40cf-ac81-e79a64223077
│      Assigned to: Risk Assessor
│      Status: ✅ Completed
│   └── 🤖 Agent: Risk Assessor
│           Status: ✅ Completed
├── 📋 Task: 06afd4af-0424-461b-a235-ff6b05a74670
│      Assigned to: Recommendation Agent
│      Status: ✅ Completed
│   └── 🤖 Agent: Recommendation Agent
│           Status: ✅ Completed
└── 📋 Task: 6951be5d-c656-4d80-8fba-cbe52aa2b665
       Status: Executing Task...
    └── 🤖 Agent: Email Agent
            Status: ✅ Completed

🚀 Crew: crew
├── 📋 Task: 168277b4-c7df-4e15-81c6-ef9e79e50b4f
│      Assigned to: Contract Reader
│      Status: ✅ Completed
│   └── 🤖 Agent: Contract Reader
│           Status: ✅ Completed
├── 📋 Task: 782fab62-6d2a-40cf-ac81-e79a64223077
│      Assigned to: Risk Assessor
│      Status: ✅ Completed
│   └── 🤖 Agent: Risk Assessor
│           Status: ✅ Completed
├── 📋 Task: 06afd4af-0424-461b-a235-ff6b05a74670
│      Assigned to: Recommendation Agent
│      Status: ✅ Completed
│   └── 🤖 Agent: Recommendation Agent
│           Status: ✅ Completed
└── 📋 Task: 6951be5d-c656-4d80-8fba-cbe52aa2b665
       Assigned to: Email Agent
       Status: ✅ Completed
    └── 🤖 Agent: Email Agent
            Status: ✅ Completed

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 6951be5d-c656-4d80-8fba-cbe52aa2b665                                                                     │
│  Agent: Email Agent                                                                                             │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: ffb91fd8-4c1b-460b-859b-4a0da0e5ab78                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

## 📬 Step 9: Final Email Report

Let’s review what the Email Agent generated as a client summary.


In [24]:
print(result)


Subject: Summary of Contract Review and Recommendations - Acme Corp and Beta Ltd.

Dear Beta Ltd.,

This email summarizes our review of the contract between Acme Corp ("Provider") and Beta Ltd ("Client"), effective January 1, 2023, concerning the provision of digital marketing services.  Our review identified several areas requiring immediate attention to mitigate potential risks and ensure compliance with internal policies.

**Summary of Findings:**

The current contract lacks sufficient detail and clarity in several key areas:

* **Vague Service Description:** The absence of specific deliverables and performance metrics for the digital marketing services creates significant uncertainty regarding the scope of work, making it susceptible to scope creep and disputes over payment and performance.  The current $5,000 monthly fee is not tied to quantifiable deliverables.

* **Indefinite Contract Term:** The ongoing nature of the contract without a defined term creates an indefinite financi